In [1]:
from dataset import SFMNNDataset
from network import SFMNNEncoder
from simulate import SFMNNSimulation
from loss import SFMNNLoss

import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import json

from dotenv import load_dotenv

/home/omirako/.local/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
load_dotenv()
data_folder = os.getenv("DATA_FOLDER")

with open(data_folder + "simulation_sim_0_amb_0.json") as f:
    data = json.load(f)
with open(data_folder + "simulation_lookuptable.json") as f:
    lookup = json.load(f)

# Load the dataset
dataset = SFMNNDataset(data_folder + 'simulation_lookuptable.json', 'output/', patch_size=5)

# Load the dataloader
dataloader = DataLoader(dataset, batch_size=3, shuffle=True, drop_last=False)


Loading JSON files:   0%|          | 0/1350 [00:00<?, ?it/s]

Loading JSON files: 100%|██████████| 1350/1350 [00:15<00:00, 85.61it/s]


Total number of elements loaded: 1350
Total number of patches loaded: 60


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# Number of spectral bands
n_spectral_bands = len(dataset.get_wl())
n_spectral_bands

3620

In [14]:
H, W, C = dataset[0].shape
H, W, C

(5, 5, 3623)

In [15]:
# Initialize the network
encoder = SFMNNEncoder(input_dim=C).to(device)

print(encoder)

SFMNNEncoder(
  (dim_red): DimReduction(
    (fc): Linear(in_features=3623, out_features=100, bias=True)
    (bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (encoder): Encoder(
    (stage1): Sequential(
      (0): ResidualMLPBlock(
        (linear): Linear(in_features=100, out_features=50, bias=True)
        (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (relu): ReLU()
        (skip): Linear(in_features=100, out_features=50, bias=True)
      )
      (1): ResidualMLPBlock(
        (linear): Linear(in_features=50, out_features=50, bias=True)
        (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (relu): ReLU()
        (skip): Identity()
      )
      (2): ResidualMLPBlock(
        (linear): Linear(in_features=50, out_features=50, bias=T

In [10]:
optimizer = optim.Adam(encoder.parameters(), lr=0.001, weight_decay=0.0001)
criterion = SFMNNLoss(torch.tensor(n_spectral_bands))

In [18]:
# test the network
for i, data in enumerate(dataloader):
    data = data.to(device)
    print('input: ' + data.shape)
    out = encoder(data)
    print('output: ' + out.shape)

TypeError: can only concatenate str (not "torch.Size") to str

In [ ]:
encoder.train()
for data in tqdm(dataloader):

    # MEASUREMENT STEP
    data = data.to(device)

    print(data.shape)

    # ENCODING STEP
    encoded = encoder(data)

    # SIMULATION STEP
    simulation = SFMNNSimulation(encoded)

    # LOSS COMPUTATION
    loss = criterion(simulation, data,)
    print(loss)

    # BACKPROPAGATION
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([3, 5, 5, 3623])


TypeError: SFMNNLoss.forward() missing 1 required positional argument: 'outputs'